## Recommendations

References:
https://salomelomsadze.medium.com/building-a-machine-learning-model-for-product-recommendations-using-customer-purchase-data-95bd86da7350

In [1]:
#Loading Libraries

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

/Users/eramkhan/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Loading and filtering required data

purchase_history = pd.read_csv('data/past_purchases.csv')
purchase_history=purchase_history[['Customer','SalesItem']].drop_duplicates()
purchase_history.rename(columns={'Customer': 'user_id', 'SalesItem': 'product_id'}, inplace=True)

In [62]:

# Count the number of purchases for each user and product combination
purchase_counts = purchase_history.groupby(['user_id', 'product_id']).size().unstack(fill_value=0)

# Convert the purchase counts to a sparse matrix
sparse_purchase_counts = sparse.csr_matrix(purchase_counts)

# Compute the cosine similarity matrix between the products
cosine_similarities = cosine_similarity(sparse_purchase_counts.T)

# Define a function to recommend items for a user based on their purchase history
def recommend_items(user_id, n=5):
    # Get the user's purchase history
    user_history = sparse_purchase_counts[user_id].toarray().flatten()

    # Compute the average cosine similarity between the user's purchased items and all other items
    similarities = cosine_similarities.dot(user_history)

    # Get the indices of the user's purchased items
    purchased_indices = np.where(user_history > 0)[0]

    # Set the similarity scores for purchased items to 0
    similarities[purchased_indices] = 0

    # Sort the items by similarity score and return the top n items
    recommended_indices = np.argsort(similarities)[::-1][:n]
    recommended_items = list(purchase_counts.columns[recommended_indices])
    
    # Remove the items that the user has already purchased
    purchased_items = list(purchase_counts.columns[purchase_counts.loc[user_id] > 0])
    recommended_items = [item for item in recommended_items if item not in purchased_items]

    return recommended_items

# Example usage:
print(recommend_items(1))  # Output: ['D', 'C', 'E']



[1248, 1245, 1470, 724, 1181]
